## Database operations in Python

After lab 5, you are able to install the Postgres/PostGIS on your machine. However, in lab5, we only used the command lines to do queries. If you want to do more complicated queries or automatic queries, you'd better to know how to do database operations through programming. This lab is about using Python to let you be able to manipulate the database table. 

## Section 1: Access and query from database

In [2]:
!pip install psycopg2

     |████████████████████████████████| 383 kB 622 kB/s eta 0:00:01
  Created wheel for psycopg2: filename=psycopg2-2.8.6-cp37-cp37m-macosx_10_9_x86_64.whl size=135462 sha256=1f8f9372f21883964ca2b8f81241de48e13b18ba6ecc89de884c5a43cd688e27
  Stored in directory: /Users/xiaojiang/Library/Caches/pip/wheels/25/78/75/9c0323f7e1fb42143cbd2439302beb7850a1034abb961cb281
Successfully built psycopg2


### 1.1 Call SQL through Python and fetch the result

In [1]:
import psycopg2

# connect your database, update pwd
conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cur = conn.cursor()

# you SQL statement
create_table_query = '''SELECT statefp, countyfp, tractce, namelsad FROM census3652;'''
cur.execute(create_table_query)

# fetch all the selected records
rows = cur.fetchall()
for row in rows:
    print("The statefp, countyfp, tractce, and namelsad", row)

conn.commit()

The statefp, countyfp, tractce, and namelsad ('42', '101', '014500', 'Census Tract 145')
The statefp, countyfp, tractce, and namelsad ('42', '101', '031200', 'Census Tract 312')
The statefp, countyfp, tractce, and namelsad ('42', '101', '021800', 'Census Tract 218')
The statefp, countyfp, tractce, and namelsad ('42', '101', '009100', 'Census Tract 91')
The statefp, countyfp, tractce, and namelsad ('42', '101', '033600', 'Census Tract 336')
The statefp, countyfp, tractce, and namelsad ('42', '101', '004202', 'Census Tract 42.02')
The statefp, countyfp, tractce, and namelsad ('42', '101', '021300', 'Census Tract 213')
The statefp, countyfp, tractce, and namelsad ('42', '101', '016800', 'Census Tract 168')
The statefp, countyfp, tractce, and namelsad ('42', '101', '007000', 'Census Tract 70')
The statefp, countyfp, tractce, and namelsad ('42', '101', '029100', 'Census Tract 291')
The statefp, countyfp, tractce, and namelsad ('42', '101', '030000', 'Census Tract 300')
The statefp, countyfp

### 1.2 Count the number of point in each census tract
We can write a SQL statement to count the number of points in each census tract and then use Python to run the SQL in database

In [2]:
import psycopg2
import fiona

conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cur = conn.cursor()


cur.execute("SELECT version();")
create_table_query = '''SELECT c2.tractce, c2.countyfp, c2.statefp, t.num, c2.geom
                        FROM (
                            SELECT c.tractce, count(p.panoid) as num
                            FROM philadata3652 p
                            LEFT JOIN census3652 c ON ST_Intersects(c.geom, p.geom)
                            GROUP BY c.tractce) t 
                        JOIN census3652 c2 ON c2. tractce = t.tractce;'''

cur.execute(create_table_query)

rows = cur.fetchall()
for row in rows:
    print("The tractce, countyfp, statefp, and number are: ", row[0], row[1], row[3])
    
conn.commit()


The tractce, countyfp, statefp, and number are:  014500 101 42
The tractce, countyfp, statefp, and number are:  031200 101 156
The tractce, countyfp, statefp, and number are:  021800 101 99
The tractce, countyfp, statefp, and number are:  009100 101 106
The tractce, countyfp, statefp, and number are:  033600 101 249
The tractce, countyfp, statefp, and number are:  004202 101 142
The tractce, countyfp, statefp, and number are:  021300 101 112
The tractce, countyfp, statefp, and number are:  016800 101 156
The tractce, countyfp, statefp, and number are:  007000 101 64
The tractce, countyfp, statefp, and number are:  029100 101 180
The tractce, countyfp, statefp, and number are:  030000 101 213
The tractce, countyfp, statefp, and number are:  000804 101 28
The tractce, countyfp, statefp, and number are:  980100 101 139
The tractce, countyfp, statefp, and number are:  013300 101 95
The tractce, countyfp, statefp, and number are:  012204 101 98
The tractce, countyfp, statefp, and number are

### 1.3 Save the result into a shapefile
Here is an example of using the Python to count the number of points in each census tract, then save the queried result to a shapefile using Fiona. You may not be able to use GeoPandas to do this. The Fiona gives you the flexibility to do much lower level operations. 

Note: Once you fetch your queried results, you have to query again if you want to fetch again. You query results can only be used once. 

In [1]:
from shapely.geometry import Point, mapping
import psycopg2
import fiona
import shapely.wkt

conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cur = conn.cursor()


cur.execute("SELECT version();")
create_table_query = '''SELECT c2.tractce, c2.countyfp, c2.statefp, t.num, c2.geom
                        FROM (
                            SELECT c.tractce, count(p.panoid) as num
                            FROM philadata3652 p
                            LEFT JOIN census3652 c ON ST_Intersects(c.geom, p.geom)
                            GROUP BY c.tractce) t 
                        JOIN census3652 c2 ON c2. tractce = t.tractce;'''

cur.execute(create_table_query)


# prepare the schema and crs of the output shapefile
schema = {
    'geometry': 'MultiPolygon',
    'properties': {
        'tractce': 'str: 20',
        'countyfp': 'str: 20',
        'count': 'int'
    }
}

crs = {'init': u'epsg:3652'}


# write the result into shapefile using fiona
with fiona.open('test.shp', 'w', driver = "ESRI Shapefile", crs = crs, schema=schema) as output:
    rows = cur.fetchall()
    for row in rows:
        print("The tractce, countyfp, statefp, and number are: ", row[0], row[1], row[3])
        tractce = row[0]
        countyfp = row[1]
        count = row[2]
        
        geom = loads(row[-1], hex = True) # <- now have a usable geometry
        
        output.write({'properties':{'tractce': tractce,
                                    'countyfp': countyfp,
                                    'count': count
                                    },
                      'geometry': mapping(geom)
                     })
        
    conn.commit()

print ('You have export the shapefile successfully')


The tractce, countyfp, statefp, and number are:  014500 101 42


NameError: name 'loads' is not defined

## Section 2 Create database/table using Python
In section 1, we only use Python to query and fetch the data from existing database table that we created in Lab 5. Now, we are going to use Python to read csv/txt files and create database and tables automatically. 


### 2.1 Create a table using Python

In [20]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")

cur = conn.cursor()

sql_statement = '''CREATE TABLE IF NOT EXISTS %s
    (PANOID               VARCHAR(30)   NOT NULL,
    YEAR                  VARCHAR(4)    NOT NULL,
    MONTH                 VARCHAR(2)    NOT NULL,
    PANO_YAW              REAL          NOT NULL,
    GVI                   REAL );
    '''%('panostreet')

cur.execute(sql_statement)
conn.commit()


### 2.2 Using this command to add a new column of geom in your table

Put this statement in your terminal or PSQL shell

`ALTER TABLE panostreet
  ADD COLUMN geom
    geometry(Geometry,3652);`

### 2.3 Let insert the records to the created database table

In [35]:
import os, os.path  
import psycopg2
import fiona
from shapely.geometry import shape

conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cursor = conn.cursor() 

pntShp = '../data/philadata3652.shp'
pntLyr = fiona.open(pntShp)
for featPnt in pntLyr:
    # get the attribute of the feature
    props = featPnt['properties']
    panoid = props['PANOID']
    year = props['YEAR']
    month = props['MONTH']
    panoyaw = props['PANO_YAW']
    gvi = props['GVI']
    
    # get the geometry of features and uwe the wkt format
    geom = shape(featPnt['geometry']).wkt
    
    query_statement = "INSERT INTO panostreet (PANOID, YEAR, MONTH, PANO_YAW, GVI, geom) VALUES('%s', '%s', '%s', '%f', '%f', ST_GeometryFromText('%s', 3652))"%(panoid, year, month, panoyaw, gvi, geom)
    cursor.execute(query_statement)  
    
conn.commit()


In [36]:
query_statement

"INSERT INTO panostreet (PANOID, YEAR, MONTH, PANO_YAW, GVI, geom) VALUES('mtm4QTzANaqakehlfXS3_A', '2011', '07', '193.000000', '8.198000', ST_GeometryFromText('POINT (2693998.922089093 242209.5879072725)', 3652))"

### 2.4. Insert records of polygons

#### 2.4.1 Create table of the polygon

In [5]:
# # create table in db
# def createTable(dbname, tbname, user, host, port):
#     '''
#     This function is used to create a table to the existing database
#     parameter:
#         dbname: the database name
        
#     '''

import psycopg2
conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")

cur = conn.cursor()

sql_statement = '''CREATE TABLE IF NOT EXISTS %s
    (statefp               VARCHAR(30)   NOT NULL,
    tractce                VARCHAR(14)    NOT NULL)
    '''%('cttable')

cur.execute(sql_statement)
conn.commit()


### Add a geometry to your database table, put this in sql shel/terminal

ALTER TABLE cttable
  ADD COLUMN geom
    geometry(Geometry,3652);

#### 2.4.2 Insert to the table records of the shapefile

In [9]:
import os, os.path  
import psycopg2
import fiona
from shapely.geometry import shape

conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cursor = conn.cursor() 

censusShp = '../data/census3652.shp'
censusLyr = fiona.open(censusShp)
for feat in censusLyr:
    # get the attribute of the feature
    props = feat['properties']
    statefp = props['STATEFP']
    tractce = props['TRACTCE']
    
    # get the geometry of features and uwe the wkt format
    geom = shape(featPnt['geometry']).wkt
    
    query_statement = "INSERT INTO cttable (statefp, tractce, geom) VALUES('%s', '%s', ST_GeometryFromText('%s', 3652))"%(statefp, tractce, geom)
    cursor.execute(query_statement)  
    
conn.commit()


## Homework:
Replicate your lab 2 using Python and database
1. Using Python to convert the shapefile into postgres tables and query you database
2. Counting of number of accidents for certain year/month for all census tract
3. save your query results into a shapefile, 
4. create a choropleth map of the accident. 

Upload your Jupyter Notebook file to Canvas. 